# 其他分析    
1. 标签关联：统计高频标签组合（词云图）
2. 字数-评分：散点图观察相关性

In [1]:
from pymongo import MongoClient
# 连接到MongoDB
client = MongoClient('mongodb://localhost:27017/')
# 打开learn数据库
db = client['learn']
# 在数据库中打开seven_cats集合
books = db['seven_cats']

In [2]:
import pandas as pd
# 将MongoDB数据转换为pandas数据
add = pd.DataFrame(list(books.find({}, {'_id': 0, 'label': 1, 'wordcount': 1, 'score': 1})))
add['count'] = 1
add

,score,label,wordcount,count
0,9.3,热血校园,272.47,1
1,9.1,古典仙侠,38.60,1
2,8.8,东方玄幻,61.96,1
3,9.3,异世大陆,664.36,1
4,9.1,东方玄幻,552.87,1
...,...,...,...,...
7713,9.3,星际机甲,329.66,1
7714,8.9,末世危机,424.60,1
7715,9.1,都市高武,135.02,1
7716,9.0,衍生同人,82.74,1


In [3]:
# 导入可视化工具
from pyecharts import options as opts

## 一、标签关联：统计高频标签组合

In [4]:
# 分组聚合
agg = pd.concat([add['label'], add['count']], axis=1)
grouped = agg.groupby('label')['count'].sum()
grouped

label
上古洪荒     117
东方玄幻    1228
乡村生活      82
传统武侠      16
侦探推理      31
其他竞技       4
历史传记       3
原生幻想      16
古典仙侠     129
商战职场     189
奇门秘术      73
官场         9
寻宝探险      31
幻想修真     194
异世大陆     456
恐怖灵异     102
战争幻想       4
抗战烽火       7
明星娱乐     112
星际机甲      17
未来幻想      71
末世危机     248
架空历史     698
武侠幻想      44
灵气复苏      63
热血校园     107
王朝争霸      23
现代军事      24
现实百态      23
电子竞技      28
穿越历史     342
篮球风云      46
虚拟网游     107
衍生同人     416
西方奇幻      20
诸天万界      85
谍战特工       2
足球天下      25
轻小说        6
都市生活     391
都市高手     960
都市高武    1169
Name: count, dtype: int64

In [5]:
# 构建词云图
from pyecharts.charts import WordCloud

In [9]:
word = WordCloud()
word.add("", [list(z) for z in zip(grouped.index, grouped)], word_size_range=[12, 66])
word.set_global_opts(
        title_opts=opts.TitleOpts(
            title="高频标签组合", title_textstyle_opts=opts.TextStyleOpts(font_size=23), 
            pos_left="center", pos_bottom="0%"
        )
    )
word.render("D:\sevencats-novels-analysis\images\七猫高频标签组合.html")

'D:\\sevencats-novels-analysis\\images\\七猫高频标签组合.html'

## 二、字数-评分：散点图观察相关性

In [10]:
agg = pd.concat([add['wordcount'], add['score']], axis=1)
sort = agg.sort_values('wordcount')
sort

,wordcount,score
4099,0.68,8.0
2379,2.00,9.2
3735,2.31,8.0
3778,2.97,8.0
2916,3.01,8.0
...,...,...
4785,2450.26,9.3
5928,2477.50,8.8
4969,2590.46,9.1
7520,2829.74,8.9


In [11]:
# 构建散点图
from pyecharts.charts import Scatter

In [12]:
st = Scatter(init_opts=opts.InitOpts(width="1440px", height="480px"))
st.add_xaxis(sort['wordcount'].tolist())
st.add_yaxis("", sort['score'].tolist(), label_opts=opts.LabelOpts(is_show=False))
st.set_global_opts(
    title_opts=opts.TitleOpts(title="字数与评分的相关性", pos_left="center", pos_bottom="2%"),
    xaxis_opts=opts.AxisOpts(name="字数（万）", type_="value", max_=5200), 
    yaxis_opts=opts.AxisOpts(name="评分", type_="value", min_=6, max_=10),
)
st.render("D:\sevencats-novels-analysis\images\七猫字数与评分的相关性.html")

'D:\\sevencats-novels-analysis\\images\\七猫字数与评分的相关性.html'